# Proyecto de SQL

Descripción: El coronavirus tomó al mundo entero por sorpresa, cambiando la rutina diaria de todos y todas. Los habitantes de las ciudades ya no pasaban su tiempo libre fuera, yendo a cafés y centros comerciales; sino que más gente se quedaba en casa, leyendo libros. Eso atrajo la atención de las startups (empresas emergentes) que se apresuraron a desarrollar nuevas aplicaciones para los amantes de los libros.

Te han dado una base de datos de uno de los servicios que compiten en este mercado. Contiene datos sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros. Esta información se utilizará para generar una propuesta de valor para un nuevo producto.

## Conexión a la BD

In [4]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Exploración inicial

In [27]:
def get_query_initial(table_name):
    query = ''' 
    SELECT *
    FROM {}
    ORDER BY 1 DESC
    LIMIT 10
    '''.format(table_name)  
    return pd.read_sql(query, con=engine)


In [ ]:
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for table in tables:
    print('----------------------------------')
    print(f'Tabla: {table}')
    print(get_query_initial(table))

----------------------------------
Tabla: books
   book_id  author_id                                              title  \
0     1000        509  Zen and the Art of Motorcycle Maintenance: An ...   
1      999         94                        You Suck (A Love Story  #2)   
2      998        201                                    Year of Wonders   
3      997        454                        Xenocide (Ender's Saga  #3)   
4      996        571           Wyrd Sisters (Discworld  #6; Witches #2)   
5      995        438                      World's End (The Sandman  #8)   
6      994        408     World War Z: An Oral History of the Zombie War   
7      993         80                                              Women   
8      992        543           Wolves of the Calla (The Dark Tower  #5)   
9      991        567                       Wolf-Speaker (Immortals  #2)   

   num_pages publication_date  publisher_id  
0        540       2006-04-25           143  
1        328       2007

## Ejercicios:

### Encuentra el número de libros publicados después del 1 de enero de 2000.

In [11]:
query = ''' 
SELECT COUNT(*)
FROM books
WHERE publication_date > '2000-01-01'
'''

pd.read_sql(query, con=engine)

,count
0,819


#### Conclusión: 
Hubo 819 libros publicados después del 1 de enero del 2000

### Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.

In [ ]:
query = ''' 
SELECT  books.title AS book_title,
        COUNT(review_id) AS number_of_reviews,
        AVG(rating) AS average_rating
FROM books
INNER JOIN reviews ON books.book_id = reviews.book_id
INNER JOIN ratings ON books.book_id = ratings.book_id
GROUP BY books.book_id
ORDER BY number_of_reviews DESC
'''

pd.read_sql(query, con=engine)


,book_title,number_of_reviews,average_rating
0,Twilight (Twilight #1),1120,3.662500
1,The Hobbit or There and Back Again,528,4.125000
2,The Catcher in the Rye,516,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,492,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,480,4.287500
...,...,...,...
989,Debt of Honor (Jack Ryan #7),2,3.000000
990,Winter Prey (Lucas Davenport #5),2,4.500000
991,Death: The High Cost of Living,2,3.000000
992,The Iliad/The Odyssey,2,4.000000


#### Conclusión:
Se hizo la agrupación conforme al ejercicio y lo ordené por el mayor número de reseñas de usuarios, teniendo para éste caso al libro de Crepúsculo en primer lugar

### Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas 

In [38]:
query = ''' 
SELECT  publisher,
        COUNT(book_id) AS number_of_books
FROM books
INNER JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE num_pages > 50
GROUP BY publisher
ORDER BY number_of_books DESC
'''

pd.read_sql(query, con=engine)


,publisher,number_of_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


#### Conclusión:
La editorial con el mayor número de libros publicados ha sido **Pinguin Books**

### Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.

In [39]:
query = ''' 
SELECT authors.author,
         AVG(rating) AS average_rating
FROM authors
INNER JOIN books ON authors.author_id = books.author_id
INNER JOIN ratings ON books.book_id = ratings.book_id
WHERE (SELECT COUNT(rating) 
       FROM ratings 
       WHERE books.book_id = ratings.book_id) >= 50
GROUP BY authors.author
ORDER BY average_rating DESC
'''

pd.read_sql(query, con=engine)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Lois Lowry,3.750000


#### Conclusión:
El Autor que tiene la más alta calificación promedio es sin lugar a dudas **J.K. Rowling/Mary GrandPré**, aunque prefiero en lo personal a J.R.R. Tolkien

### Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

In [52]:
query = ''' 
SELECT AVG(subquery.number_of_reviews) AS average_number_of_reviews
FROM (SELECT  books.book_id,
              COUNT(reviews.review_id) AS number_of_reviews
      FROM books
      INNER JOIN ratings ON books.book_id = ratings.book_id
      LEFT JOIN reviews ON books.book_id = reviews.book_id
      GROUP BY books.book_id
      HAVING COUNT(ratings.rating) > 50) AS subquery
'''

pd.read_sql(query, con=engine)


,average_number_of_reviews
0,168.755102


In [50]:
query = ''' 
SELECT MAX(subquery.number_of_reviews) AS max_number_of_reviews
FROM (SELECT  books.book_id,
              COUNT(reviews.review_id) AS number_of_reviews
      FROM books
      INNER JOIN ratings ON books.book_id = ratings.book_id
      LEFT JOIN reviews ON books.book_id = reviews.book_id
      GROUP BY books.book_id
      HAVING COUNT(ratings.rating) > 50) AS subquery
'''

pd.read_sql(query, con=engine)

,max_number_of_reviews
0,1120


In [53]:
query = ''' 
SELECT MIN(subquery.number_of_reviews) AS min_number_of_reviews
FROM (SELECT  books.book_id,
              COUNT(reviews.review_id) AS number_of_reviews
      FROM books
      INNER JOIN ratings ON books.book_id = ratings.book_id
      LEFT JOIN reviews ON books.book_id = reviews.book_id
      GROUP BY books.book_id
      HAVING COUNT(ratings.rating) > 50) AS subquery
'''

pd.read_sql(query, con=engine)

,min_number_of_reviews
0,52


#### Conclusión:

El número promedio de reseñas entre aquellos usuarios que han calificado más de 50 libros es de: **168.75**

## Conclusión
Dentro de cada uno de los ejercicios, se anotaron las conclusiones con base en lo solicitado. Las funciones de agregación, así como los JOINS son fundamentales con el fin de que, a través de dicha integración, uno sea capaz de responder a las incógnitas requeridas, junto con sus agrupaciones y condiciones correspondientes.